# LlamaParse - Fast checking Insurance Contract for Coverage

In this notebook we will look at how LlamaParse can be used to extract structured coverage information from an insurance policy.

## Installation of required packages

In [8]:
!pip install llama-index llama-parse

zsh:1: command not found: pip


## Download an insurance policy fron IRDAI

The Insurance Regulatory and Development Authority of India (IRDAI) maintains a great resource: https://policyholder.gov.in/web/guest/non-life-insurance-products where all insurance policies available in India are publicly available for download! Let's download a complex health insurance policy as an example.

In [23]:
!wget "https://policyholder.gov.in/documents/37343/931203/NBHTGBP22011V012223.pdf/c392bcc1-f6a8-cadd-ab84-495b3273d2c3?version=1.0&t=1669350459879&download=true" -O "./policy.pdf"

--2024-03-15 09:27:16--  https://policyholder.gov.in/documents/37343/931203/NBHTGBP22011V012223.pdf/c392bcc1-f6a8-cadd-ab84-495b3273d2c3?version=1.0&t=1669350459879&download=true
Resolving policyholder.gov.in (policyholder.gov.in)... 13.107.246.60, 13.107.213.60
Connecting to policyholder.gov.in (policyholder.gov.in)|13.107.246.60|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1341586 (1.3M) [application/pdf]
Saving to: ‘./policy.pdf’

./policy.pdf        100%[===================>]   1.28M  1.43MB/s    in 0.9s    

2024-03-15 09:27:17 (1.43 MB/s) - ‘./policy.pdf’ saved [1341586/1341586]



## Initializing LlamaIndex and LlamaParse

In [13]:
# llama-parse is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

import os
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."
os.environ["OPENAI_API_KEY"] = "sk-..."


In [15]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

# for the purpose of this example, we will use the small model embedding and gpt3.5
embed_model=OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-3.5-turbo-0125")

Settings.llm = llm

## Vanilla Approach - Parse the Policy with LlamaParse into Markdown

In [24]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data("./policy.pdf")

Started parsing the file under job_id 90d85533-f6d8-495d-b50c-9aab6567150d
.....

In [29]:
print(documents[0].text[0:1000])

## Preamble

This ‘Travel Infinity’ Policy is a contract of insurance between You and Us which is subject to payment of full premium in advance and the terms, conditions and exclusions of this Policy. Expense incurred outside the policy period will NOT be covered. Unutilized Sum Insured will expire at the end of the policy year. All applicable benefits, details and limits are mentioned in your Certificate of insurance. We will cover only allopathic treatments in this policy.

## Defined Terms

The terms listed below in this Section and used elsewhere in the Policy in Initial Capitals shall have the meaning set out against them in this Section.

### Standard Definitions

|2.1|Accident or Accidental|means sudden, unforeseen and involuntary event caused by external, visible and violent means.|
|---|---|---|
|2.2|Co-payment|means a cost sharing requirement under a health insurance policy that provides that the policyholder/insured will bear a specified percentage of the admissible claims a

### Markdown Element Node Parser
Our markdown element node parser works well for parsing the markdown output of LlamaParse into a set of table and text nodes.

In [26]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)

In [27]:
nodes = node_parser.get_nodes_from_documents(documents)

Embeddings have been explicitly disabled. Using MockEmbedding.


113it [00:00, 94962.20it/s]
100%|██████████| 113/113 [00:29<00:00,  3.88it/s]


In [30]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

In [35]:
query_engine = recursive_index.as_query_engine(similarity_top_k=25)

### Querying the model for coverage

In [38]:
query_1 = "My trip was delay and I paid 45, how much am I cover for?"

response_1 = query_engine.query(query_1)
print(response_1)

You are covered for the amount mentioned in the certificate of insurance for every block of hours of delay, up to a maximum of 24 hours of delay.
[NodeWithScore(node=TextNode(id_='4f9cbdb2-5079-4755-a43d-6dc15cc14082', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='573f957f-496c-496d-b59b-c0b4446e75a9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='5dbe5e3e4a2f939f5addaea005fe12421493791eceb89dbb0308d1689d4107be'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='id_c9ae03aa-9617-479e-b551-5835c074f8fb_664_table', node_type=<ObjectType.TEXT: '1'>, metadata={'table_df': "{'Benefit': {0: ' ', 1: ' '}, 'USD 100 to 500 in multiples of 100': {0: ' ', 1: 'USD 25 to 100 in multiple of 25'}, '1/2/3/5/6/100': {0: 'Independent SI', 1: '3/6/8/12'}}", 'table_summary': 'This table provides information on benefits available for an Independent SI, including the am

The information is split across the document which leads to retrieval issues. Let's try some parsing instructions to improve our result.

In [39]:
documents_with_instruction = LlamaParse(result_type="markdown", parsing_instruction="""
This document is an insurance policy.
When a benefits/coverage/exlusion is describe in the document ammend to it add a text in the follwing benefits string format (where coverage could be an exclusion).

For {nameofrisk} and in this condition {whenDoesThecoverageApply} the coverage is {coverageDescription}. 
                                        
If the document contain a benefits TABLE that describe coverage amounts, do not ouput it as a table, but instead as a list of benefits string.
                                       
""").load_data("./policy.pdf")

Started parsing the file under job_id c4e13f59-d6e8-4dd7-8d60-4bb7cfa1ad52
....

Let see how the 2 parsing compare (change target page to explore)

In [40]:
target_page = 45
pages_vanilla = documents[0].text.split("\n---\n")
pages_with_instructions = documents_with_instruction[0].text.split("\n---\n")

print(pages_vanilla[target_page])
print("\n\n=========================================================\n\n")
print(pages_with_instructions[target_page])

## Inpatient treatment

Claim Form (filled and signed by pe Insured)
Hospital Daily Cash
Release of Medical information Form (filled and signed by pe Insured)
Waiver of Deductible
Original papological and diagnostic reports, discharge summary indoor case papers (if any) and prescriptions issued by pe treating Medical practitioner or Network Provider
Optional Co-payment
Adventure Sports Cover
Home to Home Cover
Passport and Visa copy wip Entry Stamp of Country of Visit and exit Stamp from India
Extension to in-patient care
Ambulance Charge
FIR report of police (if applicable)

## Out-patient treatment

Cancer Screening & Mammographic Examination
Original bills and receipts for:
1. Charges paid towards Hospital accommodation, nursing facilities, and oper medical services rendered
2. Fees paid to pe Medical Practitioner and for special nursing charges
3. Charges incurred towards any and all test and / or examinations rendered in connection wip pe treatment
4. Charges incurred towards medi

In [42]:
node_parser_instruction = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo-0125"), num_workers=8)
nodes_instruction = node_parser.get_nodes_from_documents(documents_with_instruction)
base_nodes_instruction, objects_instruction = node_parser_instruction.get_nodes_and_objects(nodes_instruction)

recursive_index_instruction = VectorStoreIndex(nodes=base_nodes_instruction+objects_instruction)
query_engine_instruction = recursive_index_instruction.as_query_engine(similarity_top_k=25)

Embeddings have been explicitly disabled. Using MockEmbedding.


0it [00:00, ?it/s]
0it [00:00, ?it/s]


## Comparing Instruction-Augmented Parsing vs. Vanilla Parsing

When we parse the document with natural language instructions to add context on insurance coverage, we are able to correctly answer a wide range of queries in our RAG pipeline. In contrast, a RAG pipeline built with the vanilla method is not able to answer these queries.

In [44]:
query_1 = "My trip was delay and I paid 45, how much am I cover for?"

response_1 = query_engine.query(query_1)
print("Vanilla:")
print(response_1)

print("With instructions:")
response_1_i = query_engine_instruction.query(query_1)
print(response_1_i)


Vanilla:
You are covered for the amount mentioned in the certificate of insurance for every block of hours of delay, up to a maximum of 24 hours of delay.
With instructions:
You are covered for USD 500 for your delayed trip.


Looking at the policy it says in list I that one expense not covered is Baby food

In [46]:
query_2 = "I just had a baby, is baby food cover?"

response_2 = query_engine.query(query_2)
print("Vanilla:")
print(response_2)

print("With instructions:")
response_2_i = query_engine_instruction.query(query_2)
print(response_2_i)

Vanilla:
Baby food is not explicitly mentioned in the provided context information.
With instructions:
Baby food is excluded from coverage according to the provided context information.


In [51]:
query_3 = "How is gauze used in my operation coverd?"

response_3 = query_engine.query(query_3)
print("Vanilla:")
print(response_3)

print("With instructions:")
response_3_i = query_engine_instruction.query(query_3)
print(response_3_i)

Vanilla:
Gauze is not specifically mentioned in the context information provided.
With instructions:
Gauze used in your operation is covered under the benefits related to the procedure charges.
